In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
from copy import copy
from glob import glob
from PIL import Image, ImageFile
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import recgn_utils # utility script 

In [ ]:
# Verifica se CUDA está disponível 
gpu_on = torch.cuda.is_available()

if not gpu_on:
    print('Use a CPU. CUDA não está disponível...')
else:
    print('Use a GPU. CUDA está disponível...')

In [ ]:
# Configure alguns parametros:

# Path raiz do dataset 
dogs_dir = '../input/dog-breed-recognition-v3/dogs/'

# Tamanho do dataset
batch_size = 32

# Numero de workers
num_workers = 0

# Numero de epocas
num_epochs = 20

In [ ]:
# Carrega o dataset usando ImageFolder
data_dir = dogs_dir + 'train'
class_names = [item.split('/')[-2] for item in sorted(glob(data_dir + "/*/"))]

# Plota quantidade ordenada de imagens por classe, para verficar se estao desbalanceadas
_, _ = recgn_utils.check_class(data_dir)

# Calcula numero de classes
num_classes = len(class_names)

# Cria dataset de imagens     
ds_train = datasets.ImageFolder(data_dir)

# Calcula total de imagens
total_img = len(ds_train)

In [ ]:
# Define tamanho dos datasets, multiplos do batch qunado possivel
train_size = math.ceil(total_img*0.8/batch_size) * batch_size
valid_size = math.ceil(total_img*0.1/batch_size) * batch_size
test_size = total_img - (train_size + valid_size)

# Divide dataset em datasets disjuntos de treinamento, validação e teste
train_set, val_set, test_set = random_split(ds_train, [train_size, valid_size, test_size],
                                            torch.Generator().manual_seed(2147483647))

print(f'Numero de classes: {num_classes}')
print(f'Numero de imagens de treinamento: {len(train_set)}')
print(f'Numero de imagens de validacao: {len(val_set)}')
print(f'Numero de imagens de testes: {len(test_set)}')

In [ ]:
# Define valores mean e std para normalizar as imagens
# TODO: Valores baseados no ImageNet. Idealmente calcular o mean e std do dataset original
img_mean = np.array((0.485, 0.456, 0.406))
img_std = np.array((0.229, 0.224, 0.225))

# Define e aplica transformações nos datasets de treinamento, validação e teste
train_set.dataset = copy(ds_train)
train_set.dataset.transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomRotation(10),
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(img_mean, 
                                                         img_std)])
val_set.dataset.transform = transforms.Compose([transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(img_mean, 
                                                         img_std)])
test_set.dataset.transform = transforms.Compose([transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(img_mean, 
                                                         img_std)])
# Cria conjunto de loaders
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}

In [ ]:
# Exibe algumas imagens do loader com correspondentes labels

meanm = np.mean(img_mean)
stdm = np.mean(img_std)        
recgn_utils.sample_img_show(train_loader, class_names, meanm, stdm)        

In [ ]:
# Configuração do Transfer Learning

# Seleciona modelo pre-treinado
# https://pytorch.org/hub/pytorch_vision_resnet/
model = models.resnet152(pretrained=True)
print(f'Camada fully connected da ResNet152: \n{model.fc}')

# Descongela camadas para treinamento (fine-tunning)
for param in model.parameters():
    param.requires_grad = True
    
# Redefine camada fully connected
model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 512),
                                  nn.ReLU(),
                                  nn.Linear(512, 256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes),
                                  nn.LogSoftmax(dim=1)) 
print(f'\nNovo classificador: \n{model.fc}')

# Define loss function (categorical cross-entropy)
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html
criterion = nn.NLLLoss()

# Define otimizador de treinamento e diferentes taxas de aprendizado ao longo da rede
# https://pytorch.org/docs/stable/generated/torch.optim.SGD.html
ignored_params = list(map(id, model.fc.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,
                     model.parameters())
optimizer = torch.optim.SGD([
            {'params': base_params},
            {'params': model.fc.parameters(), 'lr': 0.001}
        ], lr=0.0001, momentum=0.9)

if gpu_on:
    model.cuda()

In [ ]:
# Treina modelo 

# Considera condicoes de parada:
# 1) Valid loss medio crescente
# 2) Numero de epocas = num_epochs

model = recgn_utils.train_model(model, criterion, optimizer, loaders, num_epochs, 
                                gpu_on)

In [ ]:
# Testa modelo treinado com loader de testes

prob_pass, prob_fail = recgn_utils.test_model(model, criterion, test_loader, gpu_on)

In [ ]:
# Plota distribuição de probabilidades nos cassos de pass e fail do teste   

plt.hist(prob_fail, bins = np.arange(0,1.05,0.05)) 
plt.hist(prob_pass, bins = np.arange(0,1.05,0.05), alpha = 0.7) 
labels= ["Fail","Pass"]
plt.legend(labels)
plt.xlabel('Probability')
plt.ylabel('Frequency')
plt.title('Max outputs')

In [ ]:
# Seleciona uma foto aleatoria e verifica saida do modelo

train_dir = '../input/dog-breed-recognition-v3/dogs/train/*/*'
enroll_data = np.array(glob(train_dir))
img_path = np.random.choice(enroll_data, 1)[0]
recgn_utils.imshow(img_path)
print(f'Foto selecionada aleatoriamente em:{img_path}')
pred_breed, pred_prob = recgn_utils.predict_breed_dog(model, class_names, img_mean, img_std, img_path, gpu_on)
print(f'Probabilidade de {pred_prob*100:.2f}% de ser um {pred_breed}')